**Задание**

Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше


In [35]:
!pip install stop_words pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
import numpy as np
import pandas as pd
import tensorflow as tf

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping


In [37]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [38]:
max_words = 300
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [39]:
df =  pd.read_csv('отзывы за лето.csv')
df.head(5)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [40]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [41]:
df.drop('Date', axis=1, inplace=True)
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)

In [42]:
df_train.head()

,Rating,Content
3517,5,Наконец-то исправили эту чушь с неоргинальной ...
12399,5,Удобно в использовании
3833,5,Отлично
13250,5,Класс
17941,5,Удобно


In [43]:
df_test.head()

,Rating,Content
1457,5,Всё отлично!!!
19916,5,Мне нравится. Удобно. Еще ни разу не подвел!
2112,3,Иногда слишком долго приходится ждать пока зап...
15154,5,Отличное приложение. Легко отслеживать свои ра...
18329,5,Все грамотно


In [44]:
df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [45]:
df_train.head()

,Rating,Content
3517,5,наконецтый исправить чушь снеоргинальный проши...
12399,5,удобно использование
3833,5,отлично
13250,5,класс
17941,5,удобно


In [46]:
df_test.head()

,Rating,Content
1457,5,отлично
19916,5,нравиться удобно разуна подвести
2112,3,приходиться ждать запуститься приложение бввае...
15154,5,отличный приложение легко отслеживать расход о...
18329,5,грамотно


Соберем словарь и токенизируем

In [47]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [48]:
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [49]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
len(vocabulary)

299

In [50]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [51]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)

In [52]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [53]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)

In [54]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 40, 128)           38400     
                                                                 
 conv1d_2 (Conv1D)           (None, 38, 128)           49280     
                                                                 
 activation_6 (Activation)   (None, 38, 128)           0         
                                                                 
 global_max_pooling1d_2 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 10)                1290      
                                                                 
 activation_7 (Activation)   (None, 10)                0         
                                                      

In [55]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
import tensorflow_addons as tfa

In [57]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes, average='macro')])

In [58]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])

Epoch 1/20
25/25 [==============================] - 2s 21ms/step - loss: 1.3080 - accuracy: 0.6735 - f1_score: 0.1758 - val_loss: 1.0270 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 0s 10ms/step - loss: 0.9300 - accuracy: 0.7030 - f1_score: 0.1651 - val_loss: 0.8392 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 3/20
25/25 [==============================] - 0s 9ms/step - loss: 0.7659 - accuracy: 0.7041 - f1_score: 0.2242 - val_loss: 0.7292 - val_accuracy: 0.7105 - val_f1_score: 0.2828
Epoch 4/20
25/25 [==============================] - 0s 8ms/step - loss: 0.6981 - accuracy: 0.7485 - f1_score: 0.3390 - val_loss: 0.7073 - val_accuracy: 0.7451 - val_f1_score: 0.2959
Epoch 5/20
25/25 [==============================] - 0s 8ms/step - loss: 0.6705 - accuracy: 0.7709 - f1_score: 0.3067 - val_loss: 0.7025 - val_accuracy: 0.7480 - val_f1_score: 0.2808
Epoch 6/20
25/25 [==============================] - 0s 8ms/step - loss: 0.6551 - accurac

In [59]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 1s 36ms/step - loss: 0.6786 - accuracy: 0.7672 - f1_score: 0.3550


**Поработаем с предобученным слоем**

In [56]:
!wget http://vectors.nlpl.eu/repository/20/220.zip

--2022-07-24 08:02:17--  http://vectors.nlpl.eu/repository/20/220.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638171816 (609M) [application/zip]
Saving to: ‘220.zip’

220.zip             100%[===================>] 608.61M  26.9MB/s    in 23s     

2022-07-24 08:02:41 (26.0 MB/s) - ‘220.zip’ saved [638171816/638171816]



In [57]:
!unzip 220.zip

Archive:  220.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [79]:
!pip install -U gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 496 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [60]:
import gensim

In [61]:
#загрузим векторы модели ruwikiruscorpora_upos_cbow_300_10_2021 (https://rusvectores.org/ru/models/)
word_model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)  

In [62]:
len(word_model)

249333

In [63]:
#обрежем по размеру эмбеддингов предыдущей задачи
word_model_matrix = [word_model[i][:128] for i in range(300)]

In [64]:
initializer = tf.keras.initializers.Constant(word_model_matrix)

model_w = Sequential()
model_w.add(Embedding(input_dim=max_words, output_dim=128, embeddings_initializer =initializer, input_length=max_len))
model_w.add(Conv1D(128, 3))
model_w.add(Activation("relu"))
model_w.add(GlobalMaxPool1D())
model_w.add(Dense(10))
model_w.add(Activation("relu"))
model_w.add(Dense(num_classes))
model_w.add(Activation('softmax'))
model_w.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 40, 128)           38400     
                                                                 
 conv1d_3 (Conv1D)           (None, 38, 128)           49280     
                                                                 
 activation_9 (Activation)   (None, 38, 128)           0         
                                                                 
 global_max_pooling1d_3 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 10)                1290      
                                                                 
 activation_10 (Activation)  (None, 10)                0         
                                                      

In [65]:
model_w.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes, average='macro')])

In [66]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)

history_w = model_w.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])

Epoch 1/20
25/25 [==============================] - 1s 19ms/step - loss: 1.0227 - accuracy: 0.6816 - f1_score: 0.2807 - val_loss: 0.8485 - val_accuracy: 0.7155 - val_f1_score: 0.2822
Epoch 2/20
25/25 [==============================] - 0s 10ms/step - loss: 0.7880 - accuracy: 0.7341 - f1_score: 0.3036 - val_loss: 0.8043 - val_accuracy: 0.7191 - val_f1_score: 0.3062
Epoch 3/20
25/25 [==============================] - 0s 9ms/step - loss: 0.7459 - accuracy: 0.7450 - f1_score: 0.3166 - val_loss: 0.7803 - val_accuracy: 0.7264 - val_f1_score: 0.2811
Epoch 4/20
25/25 [==============================] - 0s 9ms/step - loss: 0.7158 - accuracy: 0.7546 - f1_score: 0.3278 - val_loss: 0.7671 - val_accuracy: 0.7227 - val_f1_score: 0.2973
Epoch 5/20
25/25 [==============================] - 0s 8ms/step - loss: 0.6919 - accuracy: 0.7627 - f1_score: 0.3398 - val_loss: 0.7574 - val_accuracy: 0.7379 - val_f1_score: 0.3089
Epoch 6/20
25/25 [==============================] - 0s 9ms/step - loss: 0.6727 - accurac

In [68]:
score = model_w.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 0s 5ms/step - loss: 0.7270 - accuracy: 0.7548 - f1_score: 0.3443


**Предобученная модель показала результат немного ниже, чем модель без применения инициалайзера**